# Word Anagrams

## Introduction

**Purpose:**

The purpose of this project is to **find anagrams in the dictionary of English words**. Two words are anagrams of each other if their letters can be rearranged to turn one word into the other. For example, "$\texttt{listen}$" and "$\texttt{silent}$" are anagrams.

**Goals:**

1. Load a list of English words into a Python list.
2. Create a Python dictionary of anagrams, indexed by anagrammed word.
3. Group dictionary words by their length and then find the total number of anagrams in each group.

## Loading the English Words

We begin by loading a list of English words into Python. The data file is called `web2` and it should be located in the `../data/` folder. The file contains a list of words from *Webster's Second International Dictionary (1934)*. The 1934 copyright has lapsed and this file is included in Unix and OS X at `/usr/share/dict/web2)` as a reference word list for various uses.

Open and read in the data:

In [1]:
words = open('practice_python/data/web2', 'r').readlines()
words = [line.rstrip() for line in words]  # Remove '\n'

print(len(words))

235886


We can do the same in just one line:

In [2]:
words = [line.rstrip() for line in open('practice_python/data/web2', 'r')]

print(len(words))
words[:10]

235886


['A',
 'a',
 'aa',
 'aal',
 'aalii',
 'aam',
 'Aani',
 'aardvark',
 'aardwolf',
 'Aaron']

We should also convert all the words to lowercase since we don't need capitalized words in anagrams. We can open the file, read in its contents, remove newline characters, and change case in just one line:

In [3]:
words = [line.rstrip().lower() for line in open('practice_python/data/web2', 'r')]

print(len(words))
words[:10]

235886


['a',
 'a',
 'aa',
 'aal',
 'aalii',
 'aam',
 'aani',
 'aardvark',
 'aardwolf',
 'aaron']

Notice that the word list has duplicates, such as the terms 'A' and 'a' which now both appear as 'a' after changing the terms to lowercase. To remove duplicates, convert the list to a set and then convert back to a list. In this process, the set conversion loses the alphabetical ordering of the terms, so we also need to sort the resulting list of unique words.

In [4]:
words_unique = sorted(list(set(words)))

print(len(words_unique))
words_unique[:10]

234371


['a',
 'aa',
 'aal',
 'aalii',
 'aam',
 'aani',
 'aardvark',
 'aardwolf',
 'aaron',
 'aaronic']

**We can open the file, read in its contents, remove newline characters, change the terms to lowercase, and remove duplicates *in a single step*:**

In [5]:
words_unique = sorted(list(set([line.rstrip().lower() for line in open('practice_python/data/web2', 'r')])))

print(len(words_unique))
words_unique[:10]

234371


['a',
 'aa',
 'aal',
 'aalii',
 'aam',
 'aani',
 'aardvark',
 'aardwolf',
 'aaron',
 'aaronic']

**We can write a nice function** to accomplish the same result.

In [6]:
data_path = 'practice_python/data/web2'

# =============================================================================
def get_wordlist(data_path):
    """
    Returns a clean, sorted list of unique lowercase English words read in from
    the given data file located at the given path.
    """
    return sorted(list(set([line.rstrip().lower()
                            for line in open(data_path, 'r')])))

In [7]:
wordlist = get_wordlist(data_path)

print(len(wordlist))
wordlist[:10]

234371


['a',
 'aa',
 'aal',
 'aalii',
 'aam',
 'aani',
 'aardvark',
 'aardwolf',
 'aaron',
 'aaronic']

## Finding Anagrams

### Approach 1

Two words are anagrams of each other if their letters can be rearranged to turn one word into the other. For example, "$\texttt{listen}$" and "$\texttt{silent}$" are anagrams. Since anagrams have the same letters, just in different order, we can take advantage of the `sorted()` function and the `join()` string method to define a **word signature**. This way, **two words are anagrams of each other *if they have the same signature***.

`sorted(word)` returns a sorted list of individual word characters. Hence, two words are anagrams if `sorted(word1) == sorted(word2`.

In [8]:
sorted('listen')

['e', 'i', 'l', 'n', 's', 't']

In [9]:
sorted('listen') == sorted ('silent')

True

We use `''.join()` to join the individual characters and create the signature. We write a function to do it all in one step.

In [10]:
# =============================================================================
def signature(word):
    return ''.join(sorted(word))

We now see that "$\texttt{listen}$" and "$\texttt{silent}$" have the same signature, namely "$\texttt{eilnst}$".

In [11]:
print(signature('listen'))
print(signature('silent'))

eilnst
eilnst


Our *first approach* to finding anagrams of a given word is to go through the entire list of English words, comparing the signature of the given word against the signature of every other word in the list. This approach should not be very fast because sorting the letters of every word when creating the signature is an expensive operation $-$ `O(n log n)`. We can use the Jupyter magic `%timeit` to see how long it takes.

In [12]:
# =============================================================================
def anagrams_slow(given_word):
    return [word for word in wordlist 
            if signature(given_word) == signature(word)]

In [13]:
anagrams_slow('listen')

['enlist', 'listen', 'silent', 'tinsel']

In [14]:
%timeit anagrams_slow('listen')

246 ms ± 6.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Approach 2

Creating a signature is an expensive operation that we keep repeating for every word in the dictionary. We need a faster way to do it.

A much better approach is to create a Python dictionary of all the words indexed by their signatures. Every item in the dictionary will have a signature as the key and a list of words as the value. This way, getting the anagrams of a given word would be as simple as looking up the dictionary entry for the signature of the word and getting the list of values. This approach is `O(n)`.

We need to use a `default dict` object from the Collections module and the `append()` method to create a dictionary containing lists as values. This helps us provide a default value, in this case an empty list, to avoid a `KeyError` when you try to get a key that does not exist. For example, we get an error before the first time that we try to add the word "$\texttt{a}$" to the corresponding signature, which in this case is also "$\texttt{a}$", since the dictionary key (signature) does not exist yet.

The following simple example illustrates what is going on.

In [15]:
mydict = {}
mydict['a'] = '1'
mydict

{'a': '1'}

In [16]:
mydict['a'] = 'b'
mydict

{'a': 'b'}

Notice that simple value assignment to a key does not work for our purpose, since every time we add a value for a given key, the previous value is replaced by the new one. To keep adding values to a list for a given key, we need to use the `append()` list method.

In [18]:
mydict = {}
mydict['a'].append('1')

KeyError: 'a'

Indeed, we get a `KeyError`, as previously stated. So, we use `defaultdict` instead to initialize the dictionary.

In [19]:
import collections

In [20]:
mydict = collections.defaultdict(list)  # as opposed to mydict = {}
mydict['a'].append('1')
mydict

defaultdict(list, {'a': ['1']})

In [21]:
mydict['a'].append('b')
mydict

defaultdict(list, {'a': ['1', 'b']})

Now that we see what the situation is, we can apply this to our problem. Summarizing:

- Create an empty dictionary using `defaultdict`.
- Loop over all words in the English word list.
- Find the signature of every word, make it a key, and append the word to a list of values for the key. The value for every key (signature) becomes a list of all the words that have the same signature.

In [22]:
import collections

# =============================================================================
def anagrams_fast(word):
    return words_bysignature[signature(word)]

# Buld a dictionary of key:value pairs, where
# key:   signature(word)
# value: [words having the same signature]
words_bysignature = collections.defaultdict(list)
for word in wordlist:
    words_bysignature[signature(word)].append(word)

We can now **find anagrams by a simple dictionary lookup**.

In [23]:
anagrams_fast('listen')

['enlist', 'listen', 'silent', 'tinsel']

If we time both solutions, we see how much faster the second solution is.

In [24]:
%timeit anagrams_slow('listen')

247 ms ± 7.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%timeit anagrams_fast('listen')

540 ns ± 7.41 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Finally, we can **get all the anagrams in the list of English words**, excluding the trivial ones, where a word is an anagram of itself.

In [26]:
anagrams_wordlist = {word:anagrams_fast(word) for word in wordlist if len(anagrams_fast(word)) > 1}

In [27]:
%timeit anagrams_wordlist = {word:anagrams_fast(word) for word in wordlist if len(anagrams_fast(word)) > 1}

261 ms ± 1.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Notice that the time it takes to *find the anagrams of a single word using Approach 1* is approximately equal to the time it takes to *find the anagrams of the entire list of English words using Approach 2*. 

We can see now that about 14% of the words in the English dictionary have anagrams that are not trivial.

In [28]:
len(anagrams_wordlist)

32890

In [29]:
print("{:.2f}%".format(len(anagrams_wordlist) / len(wordlist) *100))

14.03%
